# Movielens 영화 추천 실습

## 가정
1. 유저가 평가한 별점을 시청횟수로 해석할 것
2. 3점 미만으로 준 데이터는 선호하지 않는다고 가정해 제외

## 순서
1. 데이터 준비와 전처리
2. 데이터 분석
3. 5개의 선호 영화 예시 선별
4. CSR matrix 생성
5. 모델 구성 및 훈련
6. 3의 영화 외의 영화 하나를 선별해 선호도 예측
7. 3의 영화와 비슷한 영화 추천하기
8. 가장 선호할 만한 영화 추천

### 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 미만을 받은 데이터는 제외
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings의 열 이름을 counts로 바꿈
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 확인하기 위해 메타 데이터를 읽어옴
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화 제목을 읽어온 데이터 프레임과 평점을 읽어온 데이터 프레임을 합쳐준다. 그리고 사용하지 않을 컬럼은 제거한다.

In [7]:
df = ratings.join(movies.set_index('movie_id'), on='movie_id').drop(columns=['timestamp', 'genre'])

In [8]:
df.head()

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


### 2. 데이터 분석

In [9]:
df['user_id'].nunique()

6039

In [10]:
df['movie_id'].nunique()

3628

In [11]:
# 가장 인기 많은 영화 50 개
famous_movie = df.groupby('title')['user_id'].count()
famous_movie.sort_values(ascending=False).head(50)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# user 별 통계
user_sta = df.groupby('user_id')['movie_id'].count()
user_sta.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

### 3. 5개의 선호 영화 예시 선별


In [13]:
movies[movies['title'].str.lower().str.contains('star trek', regex=False)]

,movie_id,title,genre
325,329,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
1335,1356,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi
1350,1371,Star Trek: The Motion Picture (1979),Action|Adventure|Sci-Fi
1351,1372,Star Trek VI: The Undiscovered Country (1991),Action|Adventure|Sci-Fi
1352,1373,Star Trek V: The Final Frontier (1989),Action|Adventure|Sci-Fi
1353,1374,Star Trek: The Wrath of Khan (1982),Action|Adventure|Sci-Fi
1354,1375,Star Trek III: The Search for Spock (1984),Action|Adventure|Sci-Fi
1355,1376,Star Trek IV: The Voyage Home (1986),Action|Adventure|Sci-Fi
2324,2393,Star Trek: Insurrection (1998),Action|Sci-Fi


In [14]:
example_movies_id = [329, 1356, 1371, 1372, 1374]
example_movies_title = [movies[movies['movie_id'] == i]['title'].to_list() for i in example_movies_id]

example_movies_title = list(map(lambda x: " ".join(x),example_movies_title))

In [15]:
movie_list = pd.DataFrame({'user_id': ['sample']*5, 'movie_id': example_movies_id, 'counts': [5]*5, 'title': example_movies_title})
movie_list

,user_id,movie_id,counts,title
0,sample,329,5,Star Trek: Generations (1994)
1,sample,1356,5,Star Trek: First Contact (1996)
2,sample,1371,5,Star Trek: The Motion Picture (1979)
3,sample,1372,5,Star Trek VI: The Undiscovered Country (1991)
4,sample,1374,5,Star Trek: The Wrath of Khan (1982)


In [16]:
if not df.isin({'user_id':['sample']})['user_id'].any():  # user_id에 'sample'이라는 데이터가 없다면
    df_ap_sample = df.append(movie_list, ignore_index=True)   # 위에 임의로 만든 movie_list 데이터를 추가해 줍니다. 

df_ap_sample.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,sample,329,5,Star Trek: Generations (1994)
836479,sample,1356,5,Star Trek: First Contact (1996)
836480,sample,1371,5,Star Trek: The Motion Picture (1979)
836481,sample,1372,5,Star Trek VI: The Undiscovered Country (1991)
836482,sample,1374,5,Star Trek: The Wrath of Khan (1982)


### 4. CSR Matrix 생성

In [17]:
# 고유한 유저와 영화의 개수
user_unique = df_ap_sample['user_id'].unique()
movie_unique = df_ap_sample['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [18]:
# 정상적으로 인덱싱되지 않은 경우를 제거하기 위해 dropna()
temp_user_data = df_ap_sample['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(df_ap_sample):
    print('user_id column indexing is Done')
    df_ap_sample['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing is Failed')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = df_ap_sample['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(df_ap_sample):
    print('title column indexing is Done')
    df_ap_sample['title'] = temp_movie_data
else:
    print('title column indexing is Failed')

df_ap_sample

user_id column indexing is Done
title column indexing is Done


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,329,5,558
836479,6039,1356,5,471
836480,6039,1371,5,581
836481,6039,1372,5,113


In [19]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = df_ap_sample['user_id'].nunique()
num_movie = df_ap_sample['title'].nunique()

csr_data = csr_matrix((df_ap_sample['counts'], (df_ap_sample.user_id, df_ap_sample.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5. 모델 학습

In [20]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [21]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6. 3의 영화 외의 영화 하나를 선별해 선호도 예측

In [30]:
sample, star_trek = user_to_idx['sample'], movie_to_idx['Star Trek: Generations (1994)']
sample_vector, star_trek_vector = als_model.user_factors[sample], als_model.item_factors[star_trek]

In [31]:
sample_vector

array([-0.2603937 ,  0.35270202, -0.10767446,  0.09900057,  0.15482993,
        0.4536343 , -0.4203353 , -0.1588021 ,  1.8605334 , -0.16041522,
        0.11910287, -0.75996387, -0.9672465 , -0.9177159 , -0.5771336 ,
        0.67447376, -0.23720528, -0.4189302 ,  0.00991242,  0.5993183 ,
       -0.00490461, -0.36086223, -0.19239366,  0.32690135, -0.17601301,
       -0.20445374,  0.09453163,  0.15900162, -0.25228274,  0.13202412,
        0.4390927 , -0.15848008,  0.16840556, -0.37819847,  0.14031625,
       -0.44141498,  0.55389404,  0.5949203 , -1.1472034 , -0.893363  ,
        0.62508106, -0.49156106, -0.13525644,  1.073723  ,  0.9080866 ,
       -0.158606  ,  0.2817886 ,  0.7845928 ,  0.6120548 , -0.00741457,
        0.1146834 ,  0.33680183,  0.00645449,  0.6429351 , -0.5364212 ,
        0.04776314,  0.04755144,  0.5300338 , -0.85531574,  0.17751029,
       -0.35462734,  0.31785873, -0.35880163,  0.652898  , -0.5208232 ,
        0.1904753 ,  0.30793026, -0.04646151, -0.1824187 ,  0.68

In [32]:
star_trek_vector

array([ 7.94233475e-03,  1.08492104e-02,  1.04572596e-02,  2.88640708e-02,
        1.23205334e-02,  1.94207858e-02, -9.96464677e-03, -4.55464143e-03,
        6.97001964e-02,  1.09767495e-03,  1.49011957e-02, -2.49688253e-02,
       -1.11130513e-02, -2.59009562e-02,  9.78771038e-03,  1.00147640e-02,
        1.43663436e-02, -1.18797300e-02, -5.87086761e-05,  2.54230201e-02,
        6.66650431e-03, -7.98418839e-03, -6.90558599e-03,  9.65539832e-03,
        1.39690479e-02,  1.43711884e-02,  5.85344201e-03,  6.43055048e-03,
        2.34643905e-03,  1.46619435e-02,  1.81592163e-02, -3.75997275e-03,
        1.70574617e-02, -2.76392000e-03,  1.67796630e-02, -2.77439202e-03,
        1.33357523e-02,  8.50602891e-03, -2.84978654e-02, -2.65173391e-02,
        1.92550905e-02,  4.36815573e-03,  3.98591952e-03,  2.66949162e-02,
        2.72740666e-02, -8.75313021e-03,  1.27154002e-02,  2.75768414e-02,
        1.31149720e-02,  1.00868167e-02,  9.69123840e-03,  1.75238531e-02,
       -1.15444127e-03,  

In [33]:
np.dot(sample_vector, star_trek_vector)

0.76744336

In [44]:
matrix = movie_to_idx['Star Trek V: The Final Frontier (1989)']
matrix_vector = als_model.item_factors[matrix]
np.dot(sample_vector, matrix_vector)

0.4995144

### 7. 3의 영화와 비슷한 영화 추천

In [35]:
favorite_movie = 'Star Trek: Generations (1994)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie 

[(558, 1.0),
 (113, 0.9266798),
 (699, 0.92464095),
 (471, 0.8749975),
 (584, 0.75683725),
 (1164, 0.7365231),
 (581, 0.7351268),
 (586, 0.7040624),
 (544, 0.646903),
 (583, 0.5453185),
 (1542, 0.47748575),
 (3589, 0.47629118),
 (505, 0.468798),
 (713, 0.45051083),
 (570, 0.43671912)]

In [36]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Star Trek: Generations (1994)',
 'Star Trek VI: The Undiscovered Country (1991)',
 'Star Trek: Insurrection (1998)',
 'Star Trek: First Contact (1996)',
 'Star Trek III: The Search for Spock (1984)',
 'Star Trek V: The Final Frontier (1989)',
 'Star Trek: The Motion Picture (1979)',
 'Star Trek IV: The Voyage Home (1986)',
 'Stargate (1994)',
 'Star Trek: The Wrath of Khan (1982)',
 'Timecop (1994)',
 'Soft Toilet Seats (1999)',
 'Lost in Space (1998)',
 'Starship Troopers (1997)',
 'Rocketeer, The (1991)']

In [37]:
example_movies_title

['Star Trek: Generations (1994)',
 'Star Trek: First Contact (1996)',
 'Star Trek: The Motion Picture (1979)',
 'Star Trek VI: The Undiscovered Country (1991)',
 'Star Trek: The Wrath of Khan (1982)']

리스트에 없던 Star Trek 시리즈도 추천해주는 것을 확인할 수 있었다.

In [38]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [39]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Babe (1995)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)',
 "There's Something About Mary (1998)"]

### 8. 가장 선호할 만한 영화 추천

In [40]:
user = user_to_idx['sample']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

In [41]:
movie_recommended

[(586, 0.7117504),
 (699, 0.64760864),
 (584, 0.64522207),
 (1164, 0.49951437),
 (544, 0.28241715),
 (0, 0.2580798),
 (763, 0.25086492),
 (526, 0.22406152),
 (60, 0.19433272),
 (713, 0.17975527),
 (876, 0.16280606),
 (496, 0.1592593),
 (670, 0.15246056),
 (573, 0.13376357),
 (878, 0.13124123),
 (693, 0.13114567),
 (685, 0.12953895),
 (228, 0.12834066),
 (62, 0.12771961),
 (777, 0.12710409)]

In [42]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Star Trek IV: The Voyage Home (1986)',
 'Star Trek: Insurrection (1998)',
 'Star Trek III: The Search for Spock (1984)',
 'Star Trek V: The Final Frontier (1989)',
 'Stargate (1994)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Superman II (1980)',
 'Superman (1978)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Starship Troopers (1997)',
 '2010 (1984)',
 'Alien³ (1992)',
 'Galaxy Quest (1999)',
 'M*A*S*H (1970)',
 'Strange Days (1995)',
 "Monty Python's Life of Brian (1979)",
 'Planet of the Apes (1968)',
 'X-Files: Fight the Future, The (1998)',
 'Total Recall (1990)',
 "Logan's Run (1976)"]

In [45]:
recommended = movie_to_idx['Superman II (1980)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Star Trek: The Motion Picture (1979)', 0.08056749513724251),
 ('Star Trek: The Wrath of Khan (1982)', 0.0653137622253152),
 ('Star Trek VI: The Undiscovered Country (1991)', 0.042861735401486856),
 ('Star Trek: Generations (1994)', 0.03364444829481582),
 ('Star Trek: First Contact (1996)', 0.0227090237283745)]

## 회고

데이터 전처리를 하는 과정에서 확인했을 때 영화 제목에 제작년도가 함께 들어가서 정규표현식을 통해 분리를 시도했다. 다만 이후 과정에 있어서 굳이 필요없는 작업이라고 느껴서 초기화한 뒤 생성했던 두 데이터 프레임을 합쳐서 이후 활동에 있어 용이하도록 했다. 

user_id, movie_id가 따로 있기 때문에 인덱싱을 할 필요없다고 생각했지만 새로 생성한 sample의 경우 id가 숫자로 부여되지 않았기 때문에 user과 title에 대해 다시 인덱싱을 진행했다. 

선정했던 다섯 개의 예시 영화는 스타트랙 시리즈의 일부였는데, 이렇게 선정했을 경우 비슷한 영화를 추천받는 과정에서 만약 선정되지 않았던 스타트랙 시리즈의 영화가 반환된다면 해당 모델이 유의미하게 생성되었음을 확인할 수 있다고 생각했다. 그래서 Star Trek을 포함하고 있는 영화 9개 중 5개를 선택해 예시 영화 리스트로 만들고 프로젝트를 진행했다. 

'Star Trek: Generations (1994)'와 사용자 간의 벡터를 내적했을 때, 약 0.77이라는 상대적으로 높은 결과를 기록했고, 예시 영화 리스트에 없는 'Star Trek V: The Final Frontier (1989)'의 경우 약 0.5로 이 또한 상대적으로 높은 값을 기록했다. 비슷한 영화를 추천해주는 결과를 보면 리스트에 포함되어 있던 스타트랙 시리즈 외의 것들도 포함되었음을 확인할 수 있었다.

가장 좋아하는 영화로 설정한 경우 예상대로 Star Trek의 다른 시리즈가 포함되어있을 뿐 아니라, Super man과 같이 유사한 장르의 영화를 추천해주는 것을 확인할 수 있었다. 적은 데이터 셋이지만 비교적 유의미한 결과를 반환해준다고 볼 수 있을 것 같다.